## Population Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio.mask import mask
from rasterstats import zonal_stats
import geopandas as gpd
from pathlib import Path

In [2]:
districts = gpd.read_file('./Data/district.shp')

# set CRS
districts = districts.set_crs(epsg=4326)

## Population growth

In [11]:
pop_folder = Path('./Population')

# initialize empty dataframe to store annual population data
pop_annual = districts[['NAME_2', 'geometry']].copy()
pop_annual = pop_annual.rename(columns={'NAME_2': 'district'})

# loop through each year and compute zonal stats
for year in range(2000, 2021):
    raster_path = pop_folder / f'zmb_ppp_{year}_UNadj.tif'
    # get nodata values
    with rasterio.open(raster_path) as src:
        nodata_val = src.nodata
    # compute zonal stats
    stats = zonal_stats(
        vectors=districts,
        raster=raster_path,
        stats=['sum'],
        nodata=nodata_val,
        geojson_out=False
    )
    # add to dataframe
    pop_annual[f'pop_{year}'] = [entry['sum'] for entry in stats]

# convert to long format
pop_long = pop_annual.drop(columns='geometry').melt(
    id_vars='district',
    var_name='year',
    value_name='population'
)
pop_long['year'] = pop_long['year'].str.extract(r'(\d{4})').astype(int)

pop_long = pop_long.sort_values(['district', 'year'])
pop_long['pop_growth'] = pop_long.groupby('district')['population'].pct_change()

KeyboardInterrupt: 

In [12]:
pop_long.head()

,district,year,population,pop_growth
21,Chadiza,2000,61779.734375,NaN
136,Chadiza,2001,62463.820312,0.011073
251,Chadiza,2002,64005.617188,0.024683
366,Chadiza,2003,64927.222656,0.014399
481,Chadiza,2004,66126.242188,0.018467


## Percent urban

In [21]:
import ee
ee.Authenticate()
ee.Initialize()

# load Zambia boundary
zambia = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level0') \
    .filter(ee.Filter.eq('ADM0_NAME', 'Zambia'))

# loop through years 2001-2021
for year in range(2001, 2022):
    modis = ee.ImageCollection('MODIS/006/MCD12Q1') \
        .filter(ee.Filter.calendarRange(year, year, 'year')) \
        .first() \
        .select('LC_Type1')

    # create binary urban mask
    urban_mask = modis.eq(13).selfMask().clip(zambia)
    # define export task
    task = ee.batch.Export.image.toDrive(
        image=urban_mask,
        description=f'modis_urban_mask_{year}',
        folder='GEE_Exports',
        fileNamePrefix=f'modis_urban_mask_{year}',
        region=zambia.geometry(),
        scale=500,
        crs='EPSG:4326',
        maxPixels=1e13
    )

    task.start()
    print(f'Export task for {year} started')

Export task for 2001 started
Export task for 2002 started
Export task for 2003 started
Export task for 2004 started
Export task for 2005 started
Export task for 2006 started
Export task for 2007 started
Export task for 2008 started
Export task for 2009 started
Export task for 2010 started
Export task for 2011 started
Export task for 2012 started
Export task for 2013 started
Export task for 2014 started
Export task for 2015 started
Export task for 2016 started
Export task for 2017 started
Export task for 2018 started
Export task for 2019 started
Export task for 2020 started
Export task for 2021 started
